# Automated Graph of Thoughts - Reward Shaping
This notebook demonstrates the applied reward shaping process.

## Ensure Reproducibility
The seed for the PRNG is set to $0$.

In [1]:
from stable_baselines3.common.utils import set_random_seed

SEED = 0
set_random_seed(SEED)

## Provide Required Components with Parameters
Factory function for the required components are provided.
The experiment is employed with the following parameters:
- maximum graph depth: $8$
- maximum graph breadth: $4$
- divergence cutoff factor: $0.5$

The model is trained solely on lists of length $16$.

In [2]:
from auto_graph_of_thoughts.env import GraphObservationComponent, GraphStepRewardVersion
from auto_graph_of_thoughts.experiment import ExperimentConfiguration, LanguageModelSimulationType, Experiment
from auto_graph_of_thoughts.tasks.sum_list import sum_list_task

COMPLEXITIES = [16]

base_config = ExperimentConfiguration(
        seed=SEED,
        task=sum_list_task,
        max_steps=20,
        observation_filter={
            GraphObservationComponent.DEPTH
        },
        max_depth=8,
        max_breadth=4,
        divergence_cutoff_factor=0.5,
        train_complexities=COMPLEXITIES,
        eval_complexities=COMPLEXITIES,
        max_complexity=max(COMPLEXITIES),
        max_operations=32,
        lm_simulation_type=LanguageModelSimulationType.DETERMINISTIC,
        reward_version=GraphStepRewardVersion.V0
)
base_experiment = Experiment(base_config)

## Environment
To reduce the observation space to a discrete shape, the actual environment is wrapped in a discrete observation projection wrapper.
The resulting `depth_env` is used for the training.

In [3]:
from auto_graph_of_thoughts.env.wrapper import OrdinalDiscreteToDiscreteObsMappingWrapper, \
    OrdinalDiscreteObsFilterWrapper
from auto_graph_of_thoughts.env import GraphOfThoughtsEnv, GraphObservationComponent

def create_discrete_depth_env(graph_of_thoughts_env: GraphOfThoughtsEnv):
    component = GraphObservationComponent.DEPTH
    return OrdinalDiscreteToDiscreteObsMappingWrapper(
            OrdinalDiscreteObsFilterWrapper(
                    graph_of_thoughts_env, component
            ),
            graph_of_thoughts_env.observation_space[component.value]
    )

## Set up Training
The Q-learning agent is trained on a total of $1000$ episodes with the following parameters:
- learning rate $\alpha$: $0.1$
- discount factor $\gamma$: $0.99$
- exploration rate $\epsilon$: $0.1$

In [4]:
ALPHA = 0.1
GAMMA = 0.99
EPSILON = 0.1

## Evaluation
The reward function under test is evaluated as follows.

The Q-Learning instance is trained iteratively on $100$ episodes over $10$ iterations.
After each training, a single episode is evaluated.

The results are printed for interpretation.

In [5]:
from auto_graph_of_thoughts.rl import QLearning

EPISODES_PER_ITERATION = 100
N_ITERATIONS = 10

def evaluate_reward(experiment: Experiment) -> None:
    env = experiment.create_unwrapped_train_env()
    depth_env = create_discrete_depth_env(env)
    q_learning = QLearning(depth_env, ALPHA, GAMMA, EPSILON, seed=SEED)
    for i in range(1, N_ITERATIONS+1):
        q_learning.learn(total_episodes=EPISODES_PER_ITERATION)
        terminated = False
        truncated = False
        total_rewards = 0
        state, _ = depth_env.reset()
        while not terminated and not truncated:
            action = q_learning.predict(state)
            state, reward, terminated, truncated, _ = depth_env.step(action)
            reward = float(reward)
            total_rewards += reward
        print(f'trained episodes: {i * EPISODES_PER_ITERATION}, total rewards: {total_rewards}, solved: {env.is_solved}, n operations: {env.n_operations}')

## V1: Test Sparse Reward
A simple reward function that only rewards the correct final result.
A penalty of ```-(10 / self.max_depth) * self.depth``` is applied.

In [6]:
from dataclasses import replace

r1_config = replace(base_config, reward_version=GraphStepRewardVersion.V1)
r1_experiment = Experiment(r1_config)
evaluate_reward(r1_experiment)

trained episodes: 100, total rewards: 0.0, solved: False, n operations: 0
trained episodes: 200, total rewards: 0.0, solved: False, n operations: 0
trained episodes: 300, total rewards: 0.0, solved: False, n operations: 0
trained episodes: 400, total rewards: 0.0, solved: False, n operations: 0
trained episodes: 500, total rewards: 0.0, solved: False, n operations: 0
trained episodes: 600, total rewards: 0.0, solved: False, n operations: 0
trained episodes: 700, total rewards: 0.0, solved: False, n operations: 0
trained episodes: 800, total rewards: 0.0, solved: False, n operations: 0
trained episodes: 900, total rewards: 0.0, solved: False, n operations: 0
trained episodes: 1000, total rewards: 0.0, solved: False, n operations: 0


There are no solved tasks.

## V2: Test Additional Invalid Signal
The reward function is adjusted to penalize invalid actions.

In [7]:
from dataclasses import replace

r2_config = replace(base_config, reward_version=GraphStepRewardVersion.V2)
r2_experiment = Experiment(r2_config)
evaluate_reward(r2_experiment)

trained episodes: 100, total rewards: -2.0125000000000006, solved: False, n operations: 1
trained episodes: 200, total rewards: -2.0000000000000004, solved: False, n operations: 0
trained episodes: 300, total rewards: -2.125000000000001, solved: False, n operations: 11
trained episodes: 400, total rewards: -2.0750000000000006, solved: False, n operations: 3
trained episodes: 500, total rewards: -2.2625000000000006, solved: False, n operations: 15
trained episodes: 600, total rewards: -2.0000000000000004, solved: False, n operations: 0
trained episodes: 700, total rewards: -0.625, solved: False, n operations: 11
trained episodes: 800, total rewards: -3.225, solved: False, n operations: 18
trained episodes: 900, total rewards: -1.1500000000000001, solved: False, n operations: 23
trained episodes: 1000, total rewards: -0.3375, solved: False, n operations: 3


Unfortunately, there are no solved tasks.

## V3: Test Additional Intermediate Rewards
Now, intermediate rewards are introduced.
Scored actions are now rewarded, as well as non-scored valid ones, while scored actions with a negative score are penalized.

In [8]:
from dataclasses import replace

r3_config = replace(base_config, reward_version=GraphStepRewardVersion.V3)
r3_experiment = Experiment(r3_config)
evaluate_reward(r3_experiment)

trained episodes: 100, total rewards: 1.9375000000000007, solved: False, n operations: 23
trained episodes: 200, total rewards: 1.9375000000000007, solved: False, n operations: 23
trained episodes: 300, total rewards: 1.9375000000000007, solved: False, n operations: 23
trained episodes: 400, total rewards: 1.9375000000000007, solved: False, n operations: 23
trained episodes: 500, total rewards: 1.9375000000000007, solved: False, n operations: 23
trained episodes: 600, total rewards: 1.9375000000000007, solved: False, n operations: 21
trained episodes: 700, total rewards: 1.9375000000000007, solved: False, n operations: 21
trained episodes: 800, total rewards: 1.9375000000000007, solved: False, n operations: 21
trained episodes: 900, total rewards: 1.9375000000000007, solved: False, n operations: 21
trained episodes: 1000, total rewards: 1.9375000000000007, solved: False, n operations: 21


Still, there are no solved tasks.

## V4: Test Additional Backtrack Penalty
The backtrack action indicates that there must be another append action, which should be rated negatively.

In [9]:
from dataclasses import replace

r4_config = replace(base_config, reward_version=GraphStepRewardVersion.V4)
r4_experiment = Experiment(r4_config)
evaluate_reward(r4_experiment)

trained episodes: 100, total rewards: 0.14999999999999997, solved: False, n operations: 13
trained episodes: 200, total rewards: 0.14999999999999997, solved: False, n operations: 13
trained episodes: 300, total rewards: 1.35, solved: True, n operations: 11
trained episodes: 400, total rewards: 1.35, solved: True, n operations: 9
trained episodes: 500, total rewards: 1.35, solved: True, n operations: 9
trained episodes: 600, total rewards: 1.35, solved: True, n operations: 9
trained episodes: 700, total rewards: 1.35, solved: True, n operations: 9
trained episodes: 800, total rewards: 1.35, solved: True, n operations: 9
trained episodes: 900, total rewards: 1.35, solved: True, n operations: 9
trained episodes: 1000, total rewards: 1.35, solved: True, n operations: 9


As one can see, the algorithm is able to solve the tasks after the first $300$ training episodes with $11$ operations.
After $400$ episodes, there are only $9$ operations required.

 ## V5: Test Complex Backtrack Penalty
A backtrack action must be rated less negatively if it follows a negatively scored operation.
Therefore, there is now a complex backtrack penalty in place.

In [10]:
from dataclasses import replace

r5_config = replace(base_config, reward_version=GraphStepRewardVersion.V5)
r5_experiment = Experiment(r5_config)
evaluate_reward(r5_experiment)

trained episodes: 100, total rewards: 1.35, solved: True, n operations: 11
trained episodes: 200, total rewards: -0.06250000000000006, solved: False, n operations: 13
trained episodes: 300, total rewards: 1.3375, solved: True, n operations: 9
trained episodes: 400, total rewards: 1.3375, solved: True, n operations: 9
trained episodes: 500, total rewards: 1.3375, solved: True, n operations: 9
trained episodes: 600, total rewards: 1.3375, solved: True, n operations: 9
trained episodes: 700, total rewards: 1.3375, solved: True, n operations: 9
trained episodes: 800, total rewards: 1.3375, solved: True, n operations: 9
trained episodes: 900, total rewards: 1.3375, solved: True, n operations: 9
trained episodes: 1000, total rewards: 1.3375, solved: True, n operations: 9


After $300$ episodes, the task is solved with only $9$ operations.
This is the best result so far.

## V6: Test Operation Penalty
Instead of using the depth as a penalty factor, the number of operations should be used.
This reflects the actual cost better than the depth, which might lead to a better result.

In [11]:
from dataclasses import replace

r6_config = replace(base_config, reward_version=GraphStepRewardVersion.V6)
r6_experiment = Experiment(r6_config)
evaluate_reward(r6_experiment)

trained episodes: 100, total rewards: 0.18749999999999997, solved: False, n operations: 27
trained episodes: 200, total rewards: 1.61875, solved: True, n operations: 12
trained episodes: 300, total rewards: 1.61875, solved: True, n operations: 12
trained episodes: 400, total rewards: 1.61875, solved: True, n operations: 12
trained episodes: 500, total rewards: 1.61875, solved: True, n operations: 12
trained episodes: 600, total rewards: 1.61875, solved: True, n operations: 12
trained episodes: 700, total rewards: 1.61875, solved: True, n operations: 12
trained episodes: 800, total rewards: 1.61875, solved: True, n operations: 12
trained episodes: 900, total rewards: 1.61875, solved: True, n operations: 12
trained episodes: 1000, total rewards: 1.61875, solved: True, n operations: 12


The task is solved after $200$ episodes of training.
However, there are more operations required, the number of operations stays constantly at $12$.

## Reward Function Selection
The reward function `V5` is selected as the general reward function to use.
With reward function `V5`, the Q-Learning agent required the least number of episodes for the lowest number of operations to solve the task.
For further investigation, `V4` and `V6` are kept as candidates.